# Usage With State Functionality

## Installation

In [78]:
# Uncomment the following line when running on Google Colab
# !pip install autora
# !pip install autora[synthetic]
# !pip install autora[experimentalist-pprediction-filter]

## Imports

In [96]:
from autora.state import StandardState, on_state, Delta, estimator_on_state
from autora.variable import Variable, VariableCollection
from autora.experimentalist.random import random_pool
from autora.experimentalist.prediction_filter import prediction_filter
from autora.experiment_runner.synthetic.psychology.luce_choice_ratio import luce_choice_ratio
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

## Setup: Experiment

In [80]:
experiment = luce_choice_ratio()

## Setup: State

In [81]:
state = StandardState(experiment.variables)
state.variables

VariableCollection(independent_variables=[IV(name='similarity_category_A1', value_range=(0.1, 10), allowed_values=array([ 0.1       ,  1.51428571,  2.92857143,  4.34285714,  5.75714286,
        7.17142857,  8.58571429, 10.        ]), units='similarity', type=<ValueType.REAL: 'real'>, variable_label='Similarity with Category A1', rescale=1, is_covariate=False), IV(name='similarity_category_A2', value_range=(0.1, 10), allowed_values=array([ 0.1       ,  1.51428571,  2.92857143,  4.34285714,  5.75714286,
        7.17142857,  8.58571429, 10.        ]), units='similarity', type=<ValueType.REAL: 'real'>, variable_label='Similarity with Category A2', rescale=1, is_covariate=False), IV(name='similarity_category_B1', value_range=(0.1, 10), allowed_values=array([ 0.1       ,  1.51428571,  2.92857143,  4.34285714,  5.75714286,
        7.17142857,  8.58571429, 10.        ]), units='similarity', type=<ValueType.REAL: 'real'>, variable_label='Similarity with Category B1', rescale=1, is_covariate=Fal

## Experimentalist: Random Pool on State

In [82]:
@on_state
def pool_on_state(variables, num_samples, random_state=42):
    return Delta(conditions=random_pool(
        variables=variables, 
        num_samples=num_samples, 
        random_state=random_state)
    )
state = pool_on_state(state, num_samples=100)
state.conditions

,similarity_category_A1,similarity_category_A2,similarity_category_B1,similarity_category_B2
0,0.100000,8.585714,2.928571,2.928571
1,8.585714,1.514286,10.000000,8.585714
2,7.171429,8.585714,4.342857,0.100000
3,4.342857,0.100000,7.171429,10.000000
4,4.342857,8.585714,4.342857,5.757143
...,...,...,...,...
95,2.928571,0.100000,5.757143,8.585714
96,1.514286,2.928571,8.585714,0.100000
97,7.171429,0.100000,4.342857,1.514286
98,7.171429,2.928571,2.928571,2.928571


## Experiment Runner: Run Experiment

In [83]:
@on_state
def run_on_state(conditions, random_state=42):
    return Delta(experiment_data=experiment.run(conditions, random_state=random_state))
state = run_on_state(state)
state.experiment_data

,similarity_category_A1,similarity_category_A2,similarity_category_B1,similarity_category_B2,choose_A1
0,0.100000,8.585714,2.928571,2.928571,0.010227
1,8.585714,1.514286,10.000000,8.585714,0.581342
2,7.171429,8.585714,4.342857,0.100000,0.425710
3,4.342857,0.100000,7.171429,10.000000,0.498484
4,4.342857,8.585714,4.342857,5.757143,0.279448
...,...,...,...,...,...
95,2.928571,0.100000,5.757143,8.585714,0.440030
96,1.514286,2.928571,8.585714,0.100000,0.226442
97,7.171429,0.100000,4.342857,1.514286,0.819505
98,7.171429,2.928571,2.928571,2.928571,0.620568


Theorist: Linear Regression

In [101]:
@on_state
def theorist_on_state(X, y):
    return Delta(models=[LinearRegression().fit(X, y)])
state = theorist_on_state(state)
state.models[-1]

LinearRegression()

## Filter: Prediction Filter

Let's look at predictions first:

In [102]:
conditions = random_pool(variables=experiment.variables, num_samples=4, random_state=42)
state.models[-1].predict(conditions)

array([[0.11630747],
       [0.4675618 ],
       [0.63628483],
       [0.21396789]])

Here, we want predictions that are either smaller then .2 or larger then .8 

In [103]:
filter_function = lambda x : x < .2 or x > .8

In [104]:
@on_state
def filter_on_state(conditions, models):
    filtered = prediction_filter(conditions, models[-1], filter_function)
    return Delta(conditions=filtered)
state = pool_on_state(state, num_samples=10, random_state=42)
state = filter_on_state(state)
state.conditions

,similarity_category_A1,similarity_category_A2,similarity_category_B1,similarity_category_B2
0,0.100000,5.757143,5.757143,4.342857
1,4.342857,8.585714,10.000000,5.757143
2,0.100000,5.757143,4.342857,8.585714
3,1.514286,8.585714,5.757143,2.928571
4,0.100000,4.342857,4.342857,7.171429


Let's look at the prediction to validate our filter

In [105]:
state.models[-1].predict(state.conditions)

array([[0.07178846],
       [0.1923867 ],
       [0.04447933],
       [0.07532331],
       [0.10101559]])

Let's run the experiment on these conditions and look at

In [106]:
state = run_on_state(state)
state.y[-5:]

,choose_A1
130,0.012385
131,0.256693
132,0.012034
133,0.124357
134,0.010328
